In [1]:
import numpy as np
from scipy.optimize import linprog
from ipynb.fs.full.functions import nn2na, get_selected_arcs, Aeq_TSP, arc_usage, find_arc_names
import matplotlib.pyplot as plt

Create a list of distances between cities of Argentina: 

In [2]:
dist = [[0, 344, 303, 1446, 598, 726, 859], #Azul
                [344, 0, 647, 1102, 919, 1070, 947], #B.Blanca 
                [303, 647, 0, 1720, 303, 437, 711], #Bs. As.
                [1446, 1102, 1720, 0, 2013, 2164, 1845], #C. Rivadavia
                [598, 919, 303, 2013, 0, 151, 628], #C. Uruguay
                [726, 1070, 437, 2164, 151, 0, 627], #Concordia
                [859, 947, 711, 1845, 628, 627, 0]] #Cordoba

In [3]:
node_names = ['Azul', 'B.Blanca', 'Bs. As.', 'C. Rivadavia', 'C. Uruguay', 'Concordia', 'Cordoba']

Transform array into unidimensional array and eliminate zeros (distances from a city to itself):

In [4]:
C = [item for sublist in dist for item in sublist]
for ind,arg in enumerate(C):
    if arg ==0:
        C.pop(ind)
C = np.array(C)

Create Aeq, beq and bound to use in linprog:

In [5]:
NA, arc_idxs, arc_idxs_list = nn2na(np.array(dist)) #Transform dist column into Node-Arc matrix
Aeq = Aeq_TSP(NA) # Duplicate Node-Arc matrix to have only positive values
beq = np.zeros(2 * np.shape(NA)[0]) + 1 # Necessary condition for TSP problems
bounds = tuple([(0, None) for arcs in range(0, Aeq.shape[1])]) # No upper limit for each arc

In [6]:
res = linprog(C, A_eq=Aeq, b_eq=beq, bounds=bounds, method='simplex')

C:\Users\joaquin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  """Entry point for launching an IPython kernel.


In [8]:
arc_names = find_arc_names(arc_idxs_list, node_names)
arc_use = arc_usage(arc_names, res.x.astype(int))

In [9]:
arc_use

{('Azul', 'B.Blanca'): 0,
 ('Azul', 'Bs. As.'): 1,
 ('Azul', 'C. Rivadavia'): 0,
 ('Azul', 'C. Uruguay'): 0,
 ('Azul', 'Concordia'): 0,
 ('Azul', 'Cordoba'): 0,
 ('B.Blanca', 'Azul'): 0,
 ('B.Blanca', 'Bs. As.'): 0,
 ('B.Blanca', 'C. Rivadavia'): 1,
 ('B.Blanca', 'C. Uruguay'): 0,
 ('B.Blanca', 'Concordia'): 0,
 ('B.Blanca', 'Cordoba'): 0,
 ('Bs. As.', 'Azul'): 1,
 ('Bs. As.', 'B.Blanca'): 0,
 ('Bs. As.', 'C. Rivadavia'): 0,
 ('Bs. As.', 'C. Uruguay'): 0,
 ('Bs. As.', 'Concordia'): 0,
 ('Bs. As.', 'Cordoba'): 0,
 ('C. Rivadavia', 'Azul'): 0,
 ('C. Rivadavia', 'B.Blanca'): 1,
 ('C. Rivadavia', 'Bs. As.'): 0,
 ('C. Rivadavia', 'C. Uruguay'): 0,
 ('C. Rivadavia', 'Concordia'): 0,
 ('C. Rivadavia', 'Cordoba'): 0,
 ('C. Uruguay', 'Azul'): 0,
 ('C. Uruguay', 'B.Blanca'): 0,
 ('C. Uruguay', 'Bs. As.'): 0,
 ('C. Uruguay', 'C. Rivadavia'): 0,
 ('C. Uruguay', 'Concordia'): 1,
 ('C. Uruguay', 'Cordoba'): 0,
 ('Concordia', 'Azul'): 0,
 ('Concordia', 'B.Blanca'): 0,
 ('Concordia', 'Bs. As.'): 0,
 (

### CONCLUSIONS

We see that even though the method found a solution to the problem, it created 3 sub-tours. These are:
1. Azul - Buenos Aires
2. Bahia Blanca - Comodoro Rivadavia
3. C. Uruguay - Concordia - Cordoba

These means that given this method we couldn't find ONLY 1 PATH that connects each of the locations. To solve this we could implement Dantzig's, GRASP or Genetic algorithms.